### Import các module cần thiết

In [39]:
import sqlite3, itertools, random
from collections import defaultdict

### Cấu hình tổng quát & Các tham số

In [40]:
EXAMINATION_DAYS = 24
SLOTS_PER_DAY    = 4
TIMESLOTS        = [*range(EXAMINATION_DAYS * SLOTS_PER_DAY)]

# Siêu tham số cho ACO
ALPHA = 1.0     # Ảnh hưởng của pheromone trong xác suất chọn (τ^α)
BETA  = 3.0     # Ảnh hưởng của heuristic trong xác suất chọn (η^β)
RHO   = 0.20    # Tốc độ bay hơi pheromone ở cập nhật toàn cục (global)
XI    = 0.10    # Tốc độ cập nhật pheromone cục bộ (local) sau mỗi quyết định
Q     = 1.0     # Hệ số lắng pheromone khi cộng vào lời giải tốt nhất
TAU0  = 0.1     # Giá trị pheromone khởi tạo (đồng đều)

NUM_ANTS    = 50   # Số lượng "kiến" mỗi vòng lặp
MAX_ITERS   = 30    # Số vòng lặp tối đa
EARLY_STOP  = 20   # Dừng sớm nếu không cải thiện trong chừng đó vòng
LOCAL_MOVES = 100   # Số bước thử local search cho mỗi lời giải

PENALTY: dict[str, int] = {
  "same_day_adj"   : 100000,  # Thi cùng ngày, 2 ca liền kề
  "same_day_nonadj": 7000,     # Thi cùng ngày nhưng không phải 2 ca liền kề
  "overnight_adj"  : 40,     # Thi ca cuối hôm trước và ca đầu ngày hôm sau
  "1_day"          : 30,     # Hai ca thi ở hai ngày liên tiếp
  "2_days"         : 10,     # Hai ca thi cách nhau 1 ngày
  "3_days"         : 5,      # Hai ca thi cách nhau 2 ngày
  "other"          : 0       # Các trường hợp còn lại coi như là đẹp
}

### Load data

In [41]:
with sqlite3.connect("database.db") as conn:
  cursor = conn.cursor()
  courses_by_student: defaultdict[str, set[str]] = defaultdict(set)
  sections_by_student: defaultdict[str, set[str]] = defaultdict(set)
  students_by_course: defaultdict[str, set[str]] = defaultdict(set)
  for student_id, section_code, course_code in cursor.execute("""
      SELECT e.student_id, e.section_code, s.course_code
      FROM enrolments e JOIN sections s ON e.section_code = s.section_code
  """):
    courses_by_student[student_id].add(course_code)
    sections_by_student[student_id].add(section_code)
    students_by_course[course_code].add(student_id)

### Xây dựng đồ thị xung đột và khởi tạo ma trận $\tau$
- Mỗi `course_code` là một đỉnh.
- Có cạnh giữa hai `course_code` nếu tồn tại ít nhất một sinh viên thi cả hai môn đó.
- Trọng số của cạnh là số sinh viên cùng thi hai môn đó.

In [42]:
conflict: dict[str, dict[str, int]] = defaultdict(lambda: defaultdict(int))
for course_code_list in courses_by_student.values():
  for course1, course2 in itertools.combinations(course_code_list, 2):
    conflict[course1][course2] += 1
    conflict[course2][course1] += 1
# Đảm bảo các course không có xung đột vẫn xuất hiện trong đồ thị,
# chỉ là không có cạnh nào nối tới nó.
for course in students_by_course.keys():
  conflict.setdefault(course, defaultdict(int))

tau: dict[tuple[str, int], float] = {
  (course, ts): TAU0 for course in conflict for ts in TIMESLOTS
}

### Heuristic: Tạo thứ tự xếp lịch thi
Môn nào "khó" (nhiều xung đột, quy mô lớn) được gán trước để giảm bế tắc.

In [43]:
courses_order = sorted(
  conflict.keys(),
  key=lambda c: (len(conflict[c]), len(students_by_course[c])),
  reverse=True
)

### "Vòng quay xổ số"
Chọn một phần tử trong `items` theo phân phối tỉ lệ `weights`.

In [44]:
def roulette(items: list[int], weights: list[float]):
  total = sum(weights)
  if total <= 0:
    return random.choice(items)
  r = random.random() * total
  accumulate = 0
  for it, w in zip(items, weights):
    accumulate += w
    if accumulate >= r:
      return it
  return items[-1]

### Hàm chi phí
Đánh giá độ "tốt" của một kỳ thi. Ta muốn lịch thi với mỗi sinh viên là giãn nhất có thể (nói cách khác, các môn thi không quá gần nhau).

In [45]:
def label(timeslot1: int, timeslot2: int) -> str:
  day1, slot1 = divmod(timeslot1, SLOTS_PER_DAY)
  day2, slot2 = divmod(timeslot2, SLOTS_PER_DAY)
  dist_day = day2 - day1
  if dist_day == 0:
    dist_slot = slot2 - slot1
    # Cách cài đặt thuật toán đã đảm bảo trường hợp
    # trùng ngày và trùng ca thi không thể xảy ra
    assert dist_slot != 0
    return "same_day_adj" if dist_slot == 1 else "same_day_nonadj"
  elif dist_day == 1:
    if slot1 == SLOTS_PER_DAY - 1 and slot2 == 0:
      return "overnight_adj"
    return "1_day"
  elif dist_day == 2:
    return "2_days"
  elif dist_day == 3:
    return "3_days"
  return "other"

def cost(schedule: dict[str, int]) -> tuple[int, dict[str, int]]:
  total_cost = 0
  counter = {lbl: 0 for lbl in PENALTY}    
  for courses in courses_by_student.values():
    timeslots = sorted(schedule[c] for c in courses)
    for i in range(len(timeslots) - 1):
      lbl = label(timeslots[i], timeslots[i + 1])
      total_cost += PENALTY[lbl]
      counter[lbl] += 1
  return total_cost, counter

### Xây đường đi (lịch thi) cho một con "kiến"
- Duyệt theo thứ tự `courses_order`.
- Với mỗi course: xác định tập lịch khả thi (không trùng lịch với course xung đột).
- Tính trọng số chọn lịch: $\tau^\alpha \cdot \eta^\beta$, với $\eta = \frac{1}{1+\Delta_\text{soft}}$
- Chọn lịch thi theo bánh xe xổ số, cập nhật pheromone cục bộ rồi tiếp tục.

In [46]:
def construct_ant() -> dict[str, int] | None:
  schedule: dict[str, int] = dict()
  students_by_day: dict[int, dict[str, int]] = defaultdict(lambda: defaultdict(int))
  for course in courses_order:
    forbidden_slots: set[int] = set(schedule[c] for c in conflict[course] if c in schedule)
    feasible_slots: list[int] = [ts for ts in TIMESLOTS if ts not in forbidden_slots]
    # Nếu không có slot khả thi, con kiến này "die"
    if not feasible_slots:
      return None
    weights: list[float] = []
    for timeslot in feasible_slots:
      # Heuristic cục bộ: Ước lượng nhanh "phạt tăng thêm" nếu đặt `course`
      # vào ngày thứ `day` dựa trên lịch tạm thời `partial_schedule`
      # (chỉ xét theo ngày: với mỗi sinh viên, đếm số ca thi khác trong ngày của họ).
      day = timeslot // SLOTS_PER_DAY
      delta_soft = sum(students_by_day[day][sid] for sid in students_by_course[course])
      eta = 1 / (1 + delta_soft)
      w = (tau[(course, timeslot)] ** ALPHA) * (eta ** BETA)
      weights.append(w)
    timeslot_chosen = roulette(feasible_slots, weights)
    schedule[course] = timeslot_chosen
    day_chosen = timeslot_chosen // SLOTS_PER_DAY
    for student_id in students_by_course[course]:
      students_by_day[day_chosen][student_id] += 1
    # Local pheromone update: τ ← (1-ξ)τ + ξτ0
    tau[(course, timeslot_chosen)] = (1 - XI) * tau[(course, timeslot_chosen)] + XI * TAU0
  return schedule 

### Local search: Tối ưu cục bộ đơn giản
- Lặp `LOCAL_MOVES` lần, chọn ngẫu nhiên một course, thử chuyển sang một timeslot hợp lệ khác.
- Chấp nhận nước đi nếu giảm chi phí; nếu không thì hoàn tác.

In [47]:
def local_search(schedule: dict[str, int]) -> tuple[int, dict[str, int]]:
  c, lbl = cost(schedule)
  for _ in range(LOCAL_MOVES):
    course = random.choice(courses_order)
    cur_timeslot = schedule[course]
    forbidden_slots: set[int] = set(schedule[c] for c in conflict[course] if c in schedule)
    forbidden_slots.add(cur_timeslot)
    feasible_slots: list[int] = [ts for ts in TIMESLOTS if ts not in forbidden_slots]
    if not feasible_slots:
      continue
    new_timeslot = random.choice(feasible_slots)
    schedule[course] = new_timeslot
    new_cost, new_label = cost(schedule)
    if new_cost < c:
      c = new_cost
      lbl = new_label
    else:
      schedule[course] = cur_timeslot
  return c, lbl

### Cập nhật pheromone toàn cục
- Evaporate: $\tau \leftarrow (1-\rho)\tau$ cho mọi cung `(course, timeslot)`.
- Deposit: cộng $\frac{\rho \times Q}{1 + \text{best\_cost}}$ lên các cặp `(course, timeslot)` thuộc `best_schedule`.

In [48]:
def evaporate_and_deposit(best_schedule: dict[str, int], best_cost: int) -> None:
  for course_timeslot in tau.keys():
    tau[course_timeslot] *= (1 - RHO)
  deposit = (RHO * Q) / (1 + best_cost)
  for course, timeslot in best_schedule.items():
    tau[(course, timeslot)] += deposit

### Main algorithm
Any colony optimizaztion + Local search

In [49]:
best_schedule: dict[str, int] = {}
best_labels: dict[str, int] = {}
best_cost = 99999999999
no_improvement = 0

for it in range(1, MAX_ITERS + 1):
  ants: list[dict[str, int]] = []
  labels: list[dict[str, int]] = []
  costs: list[int] = []
  while len(ants) < NUM_ANTS:
    schedule = construct_ant()
    if schedule is None:
      continue
    c, lbl = local_search(schedule)
    ants.append(schedule)
    labels.append(lbl)
    costs.append(c)
  i_best = min(range(NUM_ANTS), key=lambda i: costs[i])
  if costs[i_best] < best_cost:
    best_schedule = ants[i_best]
    best_cost = costs[i_best]
    best_labels = labels[i_best]
    no_improvement = 0
  else:
    no_improvement += 1
  evaporate_and_deposit(best_schedule, best_cost)
  print(f"[Iter {it:3d}] best_cost={best_cost:.2f} (iter_best={costs[i_best]:.2f})")
  print(labels[i_best])
  if no_improvement >= EARLY_STOP:
    print(f"No improvement for {EARLY_STOP} iterations — stop.")
    break

print(best_labels)

[Iter   1] best_cost=3315685.00 (iter_best=3315685.00)
{'same_day_adj': 24, 'same_day_nonadj': 87, 'overnight_adj': 325, '1_day': 6617, '2_days': 6487, '3_days': 6061, 'other': 12107}
[Iter   2] best_cost=3057860.00 (iter_best=3057860.00)
{'same_day_adj': 22, 'same_day_nonadj': 73, 'overnight_adj': 289, '1_day': 8570, '2_days': 5712, '3_days': 4216, 'other': 12826}
[Iter   3] best_cost=3057860.00 (iter_best=3206720.00)
{'same_day_adj': 22, 'same_day_nonadj': 98, 'overnight_adj': 411, '1_day': 6478, '2_days': 8372, '3_days': 5244, 'other': 11083}
[Iter   4] best_cost=2957360.00 (iter_best=2957360.00)
{'same_day_adj': 19, 'same_day_nonadj': 109, 'overnight_adj': 711, '1_day': 5473, '2_days': 8367, '3_days': 3612, 'other': 13417}
[Iter   5] best_cost=2957360.00 (iter_best=3076065.00)
{'same_day_adj': 22, 'same_day_nonadj': 77, 'overnight_adj': 1190, '1_day': 7137, '2_days': 5579, '3_days': 3913, 'other': 13790}
[Iter   6] best_cost=2957360.00 (iter_best=2968810.00)
{'same_day_adj': 21, 's

### Lưu lịch thi vào database

In [50]:
with sqlite3.connect("database.db") as conn:
  cursor = conn.cursor()
  cursor.executescript("""
    DROP TABLE IF EXISTS schedule;
    CREATE TABLE schedule (
      course_code  TEXT  PRIMARY KEY,
      day          INT,
      slot         INT
    )
  """)
  cursor.executemany("INSERT INTO schedule VALUES (?, ?, ?)", [(
    course,
    *divmod(timeslot, SLOTS_PER_DAY)
  ) for course, timeslot in best_schedule.items()])